In [2]:
import json
import pandas as pd

In [3]:
class Fault(object):
    def __init__(self, id, ip, iter, bit, func, latency):
        self.id = id
        self.ip = ip
        self.iter = iter
        self.bit = bit
        self.func = func
        self.latency = latency
        
    
    
    def __repr__(self):
        return str({'ID': self.id, 'IP': self.ip, 'iter': self.iter, 'bit': self.bit, 'latency': self.latency})
        print("(", self.id, self.ip, self.iter, self.bit, self.latency, ")")
        # print("(ID: %s, IP: %s, Iter: %s, Bit: %d, Latency: %d)" % (self.id, self.ip, self.iter, self.bit, self.latency))

    def __str__(self):
        return str({'ID': self.id, 'IP': self.ip, 'iter': self.iter, 'bit': self.bit, 'latency': self.latency})
    
    def __lt__(self, other):
        return self.id < other.id
        
def json_parser(ffile, filter='SIGSEGV'):
    ID = list()
    IP = list()
    IT = list()
    BIT = list()
    Func = list()
    Latency = list()
    
    with open(ffile) as fh:
        for line in fh.readlines():
            record = json.loads(line)

            status = record['status']
            if status != filter:
                continue
            
            ID.append(record['id'])
            IP.append(record['fault']['dyn_inst'][0])
            IT.append(record['fault']['dyn_inst'][1])
            BIT.append(record['fault']['bit_flipped'])
            Func.append(record['fault']['parent_func'])
            Latency.append(len(record['track']))
    d = {'ID': ID, 'IP': IP, 'Iter': IT, 'Bit': BIT, 'Func': Func, 'Latency': Latency}
    return pd.DataFrame(d)

In [22]:
ffile = '/home/cchen/Documents/Projects/CARE/Benchmarks/miniMD/O0/screen/screen_faults.json'
data = json_parser(ffile)
print(len(data))

4856


In [23]:
ranges = [0, 5, 50, 100, 400, 450]
latency = data['Latency']
dist = latency.groupby(pd.cut(latency, ranges)).count()
print(dist)

Latency
(0, 5]        4108
(5, 50]        670
(50, 100]        4
(100, 400]       2
(400, 450]      72
Name: Latency, dtype: int64


In [15]:
funcs = data['Func']
funcs.value_counts()

pushi                   1450
chargei                 1215
calc_moments             896
abs_min_int              796
chargei_init             256
radial_bin_particles     101
abs_min_real              48
poisson                   36
setup                     34
poisson_initial           21
RngStream_RandU01         20
smooth                    18
field                      4
Name: Func, dtype: int64